This notebook demonstrates how use the model api to run predictions and get information from the model. You can use it as basis to write your own experiements using the model in this container.

To start, import _modelapi.model_, which will give us access to the api of the model in this container.

In [ ]:
from modelapi import model

In [ ]:
# other helpful imports for this demo
import h5py
import json
import matplotlib.pyplot as plt
from dipy.io.image import load_nifti, save_nifti
import numpy as np

##### Various ways of handing data to the model

In [ ]:
# You can either load the necessary input for the prediction from a file:
input = 'sample_data/sample.json'
# or assemble a dictionary for yourself 
input_dict = {"t1":{},"t2":{},"flair":{},"t1c":{}}
# mandatory: fileurl for each input!
input_dict["t1"]["fileurl"] = 'sample_data/patient_1/t1.nii.gz'
# mandatory: format for each input, be aware that this has to be a list:
input_dict["t1"]["format"] = ["application/nii-gzip"]
# and so on for each other input.. 

In [ ]:
with open(input, "r") as f:
    input_dict_from_json = json.load(f)

In [ ]:
# you can then simply pass your input to the model:
result = model.predict(input_dict_from_json)

In [ ]:
# let's see what the output contains:
result

##### Recover a prediction from the h5 file

In [ ]:
# recover prediction from h5 file
resultFile = result["output"][0]["prediction"].replace("api", "")
f = h5py.File(resultFile, 'r')
print(list(f.keys()))
# load the actual prediction and check for correct shape
prediction = f["Segmentation"]
print('Shape of prediction is: {}'.format(prediction.shape))

In [ ]:
# just plot one slice to see if it returns a sensible result 
plt.imshow(prediction[90])
print(prediction[90].max(), prediction[90].min())

##### Saving the result as a Nifti-1 file so we can use it

In [ ]:
# first, we have to swap the axes of the array to match the shape of the original images (155,240,240) -> (240,240,155)
#prediction = np.swapaxes(prediction, 0,2)
# second, we load the metadata from one of the input images to populate the header:
import nibabel as nib 
img = nib.load('sample_data/patient_1/flair.nii.gz')
header = img.header
affine = img.affine
seg_nifti = nib.Nifti1Image(prediction, affine, header)
nib.save(seg_nifti,'sample_data/patient_1/prediction.nii.gz')

If the model does not provide any sample data, print an error message (this part also shows how to read meta info - like the model's name - from the models config).